In [1]:
import numpy as np

# 1. Environment and Algorithm Parameters

In [2]:
GRID_SIZE = 4
TERMINAL_STATES = [0, GRID_SIZE * GRID_SIZE - 1]
GAMMA = 1.0
REWARD = -1.0

# 2. Helper Functions

In [3]:
def get_greedy_policy(value_function):
    """Computes the greedy policy based on the value function."""
    # policy is a 16x4 array for 16 states and 4 actions
    # Actions are indexed 0:UP, 1:DOWN, 2:LEFT, 3:RIGHT
    policy = np.zeros((GRID_SIZE * GRID_SIZE, 4)) 
    for state in range(GRID_SIZE * GRID_SIZE):
        if state in TERMINAL_STATES:
            continue
        
        action_values = []
        # UP
        next_state = state - GRID_SIZE if state >= GRID_SIZE else state
        action_values.append(REWARD + GAMMA * value_function[next_state])
        # DOWN
        next_state = state + GRID_SIZE if state < GRID_SIZE * (GRID_SIZE - 1) else state
        action_values.append(REWARD + GAMMA * value_function[next_state])
        # LEFT
        next_state = state - 1 if state % GRID_SIZE != 0 else state
        action_values.append(REWARD + GAMMA * value_function[next_state])
        # RIGHT
        next_state = state + 1 if (state + 1) % GRID_SIZE != 0 else state
        action_values.append(REWARD + GAMMA * value_function[next_state])

        best_action_value = np.max(action_values)
        # Use a small tolerance for floating point comparison to find all 'best' actions
        best_actions = np.where(np.abs(action_values - best_action_value) < 1e-6)[0]
        for action in best_actions:
            policy[state, action] = 1 # Mark best actions with 1
            
    return policy

In [7]:
def pretty_print(v, policy, k, delta):
    """Prints the value function and policy in a formatted grid."""
    print(f"k={k}")
    if k != 0:
        print(f"delta: {delta:.6f}\n")

    # --- Print Value Function ---
    v_grid = v.reshape(GRID_SIZE, GRID_SIZE)
    for i in range(GRID_SIZE):
        row_str = " | ".join([f"{val:06.2f}" for val in v_grid[i]])
        print(f"{row_str} |")
    
    print("-" * (GRID_SIZE * 9)) # Separator line

    # --- Print Policy ---
    arrows = ['↑', '↓', '←', '→']
    for i in range(GRID_SIZE):
        row_str = ""
        for j in range(GRID_SIZE):
            state = i * GRID_SIZE + j
            if state in TERMINAL_STATES:
                cell_str = " "
            else:
                action_indices = np.where(policy[state] == 1)[0]
                cell_str = "".join([arrows[idx] for idx in action_indices])
            # ljust pads the string with spaces to ensure uniform column width
            row_str += f"{cell_str.ljust(4)} | "
        print(row_str)
    
    print("\n" + "="*40 + "\n")

# 3. Main Policy Evaluation Algorithm

In [8]:
def run_policy_evaluation():
    """
    Performs iterative policy evaluation and prints the results at each step.
    """
    v = np.zeros(GRID_SIZE * GRID_SIZE)
    
    # --- Print initial state (k=0) ---
    policy = get_greedy_policy(v)
    pretty_print(v, policy, 0, 0)
    
    iteration = 0
    while True:
        iteration += 1
        delta = 0
        
        # Loop through each state for an in-place update
        for state in range(GRID_SIZE * GRID_SIZE):
            if state in TERMINAL_STATES:
                continue
                
            old_value = v[state]
            new_value = 0.0
            action_prob = 0.25 # Random policy
            
            # UP
            next_state = state - GRID_SIZE if state >= GRID_SIZE else state
            new_value += action_prob * (REWARD + GAMMA * v[next_state])
            # DOWN
            next_state = state + GRID_SIZE if state < GRID_SIZE * (GRID_SIZE - 1) else state
            new_value += action_prob * (REWARD + GAMMA * v[next_state])
            # LEFT
            next_state = state - 1 if state % GRID_SIZE != 0 else state
            new_value += action_prob * (REWARD + GAMMA * v[next_state])
            # RIGHT
            next_state = state + 1 if (state + 1) % GRID_SIZE != 0 else state
            new_value += action_prob * (REWARD + GAMMA * v[next_state])
            
            v[state] = new_value
            delta = max(delta, abs(old_value - v[state]))
        
        # --- Print intermediate results ---
        if iteration in [1, 2, 3, 10]:
            policy = get_greedy_policy(v)
            pretty_print(v, policy, iteration, delta)
            
        # --- Check for convergence ---
        if delta < 1e-4:
            policy = get_greedy_policy(v)
            print("Converged!")
            pretty_print(v, policy, '∞', delta)
            break

# 4. Run the Experiment

In [9]:
run_policy_evaluation()

k=0
000.00 | 000.00 | 000.00 | 000.00 |
000.00 | 000.00 | 000.00 | 000.00 |
000.00 | 000.00 | 000.00 | 000.00 |
000.00 | 000.00 | 000.00 | 000.00 |
------------------------------------
     | ↑↓←→ | ↑↓←→ | ↑↓←→ | 
↑↓←→ | ↑↓←→ | ↑↓←→ | ↑↓←→ | 
↑↓←→ | ↑↓←→ | ↑↓←→ | ↑↓←→ | 
↑↓←→ | ↑↓←→ | ↑↓←→ |      | 


k=1
delta: 1.898438

000.00 | -01.00 | -01.25 | -01.31 |
-01.00 | -01.50 | -01.69 | -01.75 |
-01.25 | -01.69 | -01.84 | -01.90 |
-01.31 | -01.75 | -01.90 | 000.00 |
------------------------------------
     | ←    | ←    | ←    | 
↑    | ↑←   | ↑    | ↑    | 
↑    | ←    | ↑←   | ↓    | 
↑    | ←    | →    |      | 


k=2
delta: 1.724609

000.00 | -01.94 | -02.55 | -02.73 |
-01.94 | -02.81 | -03.24 | -03.40 |
-02.55 | -03.24 | -03.57 | -03.22 |
-02.73 | -03.40 | -03.22 | 000.00 |
------------------------------------
     | ←    | ←    | ←    | 
↑    | ↑←   | ↑    | ↑    | 
↑    | ←    | ↓→   | ↓    | 
↑    | ←    | →    |      | 


k=3
delta: 1.472412

000.00 | -02.82 | -03.83 | -04.18 |
